In [2]:
import googlemaps
# Careful. This costs $$$
GOOGLE_API_KEY='GOOGLE_API_KEY'
if GOOGLE_API_KEY not in os.environ:
    raise Exception(f"Must set {GOOGLE_API_KEY} environment variable to call Google's APIs.")

In [3]:
import os
gmaps = googlemaps.Client(os.environ[GOOGLE_API_KEY])

In [24]:
# Riverbend Church
origin = '4214 N Capital of Texas Hwy, Austin, TX 78746'
origin_lat = 30.3418005
origin_lng = -97.8040886

In [6]:
import pandas as pd
customers = pd.read_csv('customers.csv')
customers

,name,address,bags
0,Mopac,"10515 N Mopac Expy, Austin, TX 78759",16
1,Lake Travis,"7900 N FM 620, Austin, TX 78726",39
2,Lakeline,"11301 Lakeline Blvd, Austin, TX 78717",33
3,Cedar Park,"2700 E Whitestone Blvd, Cedar Park, TX 78613",38
4,Pflugerville,"2551 S N Interstate Hwy 35, Round Rock, TX 78664",9
5,Wells Branch,"13309 N Interstate Hwy 35 North, Austin, TX 78753",35
6,Mueller,"1200 Barbara Jordan Blvd Suite 100, Austin, TX...",10
7,Sunset Valley,"1200 Home Depot Blvd, Sunset Valley, TX 78745",26
8,St. Edwards,"3600 S IH 35 Frontage Rd S, Austin, TX 78704",25


In [7]:
latlong=customers.address.map(gmaps.geocode)
latlong

0    [{'address_components': [{'long_name': '10515'...
1    [{'address_components': [{'long_name': '7900',...
2    [{'address_components': [{'long_name': '11301'...
3    [{'address_components': [{'long_name': '2700',...
4    [{'address_components': [{'long_name': '2551 S...
5    [{'address_components': [{'long_name': '13309'...
6    [{'address_components': [{'long_name': '100', ...
7    [{'address_components': [{'long_name': '1200',...
8    [{'address_components': [{'long_name': '3600',...
Name: address, dtype: object

In [20]:
customers['lat'] = [response[0]['geometry']['location']['lat'] for response in latlong]
customers['lng'] = [response[0]['geometry']['location']['lng'] for response in latlong]
customers

,name,address,bags,lat,lng
0,Mopac,"10515 N Mopac Expy, Austin, TX 78759",16,30.390073,-97.732630
1,Lake Travis,"7900 N FM 620, Austin, TX 78726",39,30.414619,-97.850905
2,Lakeline,"11301 Lakeline Blvd, Austin, TX 78717",33,30.478590,-97.798990
3,Cedar Park,"2700 E Whitestone Blvd, Cedar Park, TX 78613",38,30.531053,-97.788325
4,Pflugerville,"2551 S N Interstate Hwy 35, Round Rock, TX 78664",9,30.485073,-97.673493
5,Wells Branch,"13309 N Interstate Hwy 35 North, Austin, TX 78753",35,30.421818,-97.668922
6,Mueller,"1200 Barbara Jordan Blvd Suite 100, Austin, TX...",10,30.306521,-97.708218
7,Sunset Valley,"1200 Home Depot Blvd, Sunset Valley, TX 78745",26,30.228748,-97.824776
8,St. Edwards,"3600 S IH 35 Frontage Rd S, Austin, TX 78704",25,30.222166,-97.749652


In [48]:
import folium

def origin_marker(lat, lng):
    return folium.Marker(location=[lat, lng],
                         icon=folium.Icon(icon='star', color='blue'))

def customer_marker(lat, lng, bags):
    return folium.Marker(location=[lat, lng],
                         tooltip=str(bags),
                         icon=folium.Icon(icon='user', color='green'))

map = folium.Map(location=[origin_lat, origin_lng], default_zoom_start=17)
origin_marker(origin_lat, origin_lng).add_to(map)

for _, row in customers.iterrows():
    customer_marker(row.lat, row.lng, row.bags).add_to(map)
map

In [53]:
response = gmaps.distance_matrix(origins=[origin],
                                 destinations=customers.address,
                                 mode='driving',
                                 avoid='tolls')

In [54]:
for dst, el in zip(customers.name, response['rows'][0]['elements']):
    print(f"Riverbend to {dst}: {el['duration']['text']}")

Riverbend to Mopac: 15 mins
Riverbend to Lake Travis: 14 mins
Riverbend to Lakeline: 17 mins
Riverbend to Cedar Park: 29 mins
Riverbend to Pflugerville: 25 mins
Riverbend to Wells Branch: 22 mins
Riverbend to Mueller: 18 mins
Riverbend to Sunset Valley: 17 mins
Riverbend to St. Edwards: 19 mins
